In [ ]:
%%capture
!pip install -U stanza conllu==4.5 tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
!wget -O eval.py https://raw.githubusercontent.com/UniversalDependencies/tools/refs/heads/master/eval.py
!chmod +x eval.py

In [ ]:
import stanza, pathlib
stanza.download('sv')

In [ ]:
!wget -O sv_talbanken.tar.gz https://github.com/UniversalDependencies/UD_Swedish-Talbanken/archive/refs/tags/r2.16.tar.gz
!tar -xf sv_talbanken.tar.gz

In [ ]:
from stanza.utils.conll import CoNLL
from tqdm import tqdm
import stanza, pathlib

nlp = stanza.Pipeline(
        lang='sv',
        processors='tokenize,pos,lemma,depparse',
        tokenize_pretokenized=False,
        use_gpu=False)

In [ ]:
from conllu import parse_incr

for split in ["test", "dev", "train"]:
    INFILE=f"UD_Swedish-Talbanken-r2.16/sv_talbanken-ud-{split}.conllu"
    OUTFILE = f"talbanken-stanza-{split}.conllu"

    with open(INFILE, encoding="utf-8") as gold,\
        open(OUTFILE,  "w", encoding="utf-8") as out:

        for tokenlist in tqdm(parse_incr(gold)):
            doc = nlp(tokenlist.metadata["text"])

            lines = "{:C}".format(doc).split("\n")
            lines = [line for line in lines if not line.startswith("#")]

            mdlines = []
            for mditem in tokenlist.metadata:
                mdlines.append(f"# {mditem}: {tokenlist.metadata[mditem]}")
            lines = mdlines + lines
            out.write("\n".join(lines))
            out.write("\n\n")
